# Getting started with Word2Vec in Gensim and making it work!

The idea behind Word2Vec is pretty simple. We are making and assumption that you can tell the meaning of a word by the company it keeps. This is analogous to the saying *show me your friends, and I'll tell who you are*. So if you have two words that have very similar neighbors (i.e. the usage context is about the same), then these words are probably quite similar in meaning or are at least highly related. For example, the words `shocked`,`appalled` and `astonished` are typically used in a similar context. 

In this tutorial, you will learn how to use the Gensim implementation of Word2Vec and actually get it to work! I have heard a lot of complaints about poor performance etc, but its really a combination of two things, (1) your input data and (2) your parameter settings. Note that the training algorithms in this package were ported from the [original Word2Vec implementation by Google](https://arxiv.org/pdf/1301.3781.pdf) and extended with additional functionality.

### Imports and logging

First, we start with our imports and get logging established:

In [9]:
# imports needed and set up logging
import gzip
import gensim 
import logging

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)


### Dataset 
Next, is our dataset. The secret to getting Word2Vec really working for you is to have lots and lots of text data. In this case I am going to use data from the [OpinRank](http://kavita-ganesan.com/entity-ranking-data/) dataset. This dataset has full user reviews of cars and hotels. I have specifically concatenated all of the hotel reviews into one big file which is about 97MB compressed and 229MB uncompressed. We will use the compressed file for this tutorial. Each line in this file represents a hotel review. You can download the OpinRank Word2Vec dataset here.

To avoid confusion, while gensim’s word2vec tutorial says that you need to pass it a sequence of sentences as its input, you can always pass it a whole review as a sentence (i.e. a much larger size of text), and it should not make much of a difference. 

Now, let's take a closer look at this data below by printing the first line. You can see that this is a pretty hefty review.

In [13]:
data_file="data/reviews_data.txt.gz"

with gzip.open ('data/reviews_data.txt.gz', 'rb') as f:
    for i,line in enumerate (f):
        print(line)
        break


b"Oct 12 2009 \tNice trendy hotel location not too bad.\tI stayed in this hotel for one night. As this is a fairly new place some of the taxi drivers did not know where it was and/or did not want to drive there. Once I have eventually arrived at the hotel, I was very pleasantly surprised with the decor of the lobby/ground floor area. It was very stylish and modern. I found the reception's staff geeting me with 'Aloha' a bit out of place, but I guess they are briefed to say that to keep up the coroporate image.As I have a Starwood Preferred Guest member, I was given a small gift upon-check in. It was only a couple of fridge magnets in a gift box, but nevertheless a nice gesture.My room was nice and roomy, there are tea and coffee facilities in each room and you get two complimentary bottles of water plus some toiletries by 'bliss'.The location is not great. It is at the last metro stop and you then need to take a taxi, but if you are not planning on going to see the historic sites in Be

### Read files into a list
Now that we've had a sneak peak of our dataset, we can read it into a list so that we can pass this on to the Word2Vec model. Notice in the code below, that I am directly reading the 
compressed file. I'm also doing a mild pre-processing of the reviews using `gensim.utils.simple_preprocess (line)`. This does some basic pre-processing such as tokenization, lowercasing, etc and returns back a list of tokens (words). Documentation of this pre-processing method can be found on the official [Gensim documentation site](https://radimrehurek.com/gensim/utils.html). 



In [14]:

def read_input(input_file):
    """This method reads the input file which is in gzip format"""
    
    logging.info("reading file {0}...this may take a while".format(input_file))
    
    with gzip.open (input_file, 'rb') as f:
        for i, line in enumerate (f): 

            if (i%10000==0):
                logging.info ("read {0} reviews".format (i))
            # do some pre-processing and return a list of words for each review text
            yield gensim.utils.simple_preprocess (line)

# read the tokenized reviews into a list
# each review item becomes a serries of words
# so this becomes a list of lists
documents = list (read_input (data_file))
logging.info ("Done reading data file")    

2019-07-14 13:21:06,506 : INFO : reading file data/reviews_data.txt.gz...this may take a while
2019-07-14 13:21:06,507 : INFO : read 0 reviews
2019-07-14 13:21:07,830 : INFO : read 10000 reviews
2019-07-14 13:21:09,139 : INFO : read 20000 reviews
2019-07-14 13:21:10,647 : INFO : read 30000 reviews
2019-07-14 13:21:12,062 : INFO : read 40000 reviews
2019-07-14 13:21:13,614 : INFO : read 50000 reviews
2019-07-14 13:21:15,336 : INFO : read 60000 reviews
2019-07-14 13:21:16,621 : INFO : read 70000 reviews
2019-07-14 13:21:17,788 : INFO : read 80000 reviews
2019-07-14 13:21:19,045 : INFO : read 90000 reviews
2019-07-14 13:21:20,256 : INFO : read 100000 reviews
2019-07-14 13:21:21,502 : INFO : read 110000 reviews
2019-07-14 13:21:22,706 : INFO : read 120000 reviews
2019-07-14 13:21:23,940 : INFO : read 130000 reviews
2019-07-14 13:21:25,267 : INFO : read 140000 reviews
2019-07-14 13:21:26,530 : INFO : read 150000 reviews
2019-07-14 13:21:28,218 : INFO : read 160000 reviews
2019-07-14 13:21:2

## Training the Word2Vec model

Training the model is fairly straightforward. You just instantiate Word2Vec and pass the reviews that we read in the previous step (the `documents`). So, we are essentially passing on a list of lists. Where each list within the main list contains a set of tokens from a user review. Word2Vec uses all these tokens to internally create a vocabulary. And by vocabulary, I mean a set of unique words.

After building the vocabulary, we just need to call `train(...)` to start training the Word2Vec model. Training on the [OpinRank](http://kavita-ganesan.com/entity-ranking-data/) dataset takes about 10 minutes so please be patient while running your code on this dataset.

Behind the scenes we are actually training a simple neural network with a single hidden layer. But, we are actually not going to use the neural network after training. Instead, the goal is to learn the weights of the hidden layer. These weights are essentially the word vectors that we’re trying to learn. 

In [15]:
model = gensim.models.Word2Vec (documents, size=150, window=10, min_count=2, workers=10)
model.train(documents,total_examples=len(documents),epochs=10)

2019-07-14 13:21:42,109 : WARNING : consider setting layer size to a multiple of 4 for greater performance
2019-07-14 13:21:42,109 : INFO : collecting all words and their counts
2019-07-14 13:21:42,110 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-07-14 13:21:42,294 : INFO : PROGRESS: at sentence #10000, processed 1655714 words, keeping 25777 word types
2019-07-14 13:21:42,475 : INFO : PROGRESS: at sentence #20000, processed 3317863 words, keeping 35016 word types
2019-07-14 13:21:42,681 : INFO : PROGRESS: at sentence #30000, processed 5264072 words, keeping 47518 word types
2019-07-14 13:21:42,876 : INFO : PROGRESS: at sentence #40000, processed 7081746 words, keeping 56675 word types
2019-07-14 13:21:43,087 : INFO : PROGRESS: at sentence #50000, processed 9089491 words, keeping 63744 word types
2019-07-14 13:21:43,293 : INFO : PROGRESS: at sentence #60000, processed 11013726 words, keeping 76786 word types
2019-07-14 13:21:43,480 : INFO : PROGRESS: a

2019-07-14 13:22:15,884 : INFO : EPOCH 2 - PROGRESS: at 7.75% examples, 1196186 words/s, in_qsize 19, out_qsize 0
2019-07-14 13:22:16,887 : INFO : EPOCH 2 - PROGRESS: at 11.24% examples, 1208001 words/s, in_qsize 19, out_qsize 0
2019-07-14 13:22:17,897 : INFO : EPOCH 2 - PROGRESS: at 14.16% examples, 1164174 words/s, in_qsize 19, out_qsize 0
2019-07-14 13:22:18,906 : INFO : EPOCH 2 - PROGRESS: at 16.61% examples, 1101252 words/s, in_qsize 20, out_qsize 0
2019-07-14 13:22:19,914 : INFO : EPOCH 2 - PROGRESS: at 19.26% examples, 1080198 words/s, in_qsize 19, out_qsize 0
2019-07-14 13:22:20,932 : INFO : EPOCH 2 - PROGRESS: at 22.54% examples, 1089038 words/s, in_qsize 16, out_qsize 3
2019-07-14 13:22:21,941 : INFO : EPOCH 2 - PROGRESS: at 25.69% examples, 1088294 words/s, in_qsize 18, out_qsize 1
2019-07-14 13:22:22,957 : INFO : EPOCH 2 - PROGRESS: at 29.49% examples, 1080209 words/s, in_qsize 16, out_qsize 3
2019-07-14 13:22:23,966 : INFO : EPOCH 2 - PROGRESS: at 33.23% examples, 1075097 

2019-07-14 13:23:16,810 : INFO : worker thread finished; awaiting finish of 5 more threads
2019-07-14 13:23:16,819 : INFO : worker thread finished; awaiting finish of 4 more threads
2019-07-14 13:23:16,826 : INFO : worker thread finished; awaiting finish of 3 more threads
2019-07-14 13:23:16,828 : INFO : worker thread finished; awaiting finish of 2 more threads
2019-07-14 13:23:16,831 : INFO : worker thread finished; awaiting finish of 1 more threads
2019-07-14 13:23:16,838 : INFO : worker thread finished; awaiting finish of 0 more threads
2019-07-14 13:23:16,839 : INFO : EPOCH - 3 : training on 41519358 raw words (30348494 effective words) took 32.1s, 945095 effective words/s
2019-07-14 13:23:17,855 : INFO : EPOCH 4 - PROGRESS: at 2.52% examples, 786797 words/s, in_qsize 16, out_qsize 3
2019-07-14 13:23:18,865 : INFO : EPOCH 4 - PROGRESS: at 5.62% examples, 857565 words/s, in_qsize 19, out_qsize 0
2019-07-14 13:23:19,865 : INFO : EPOCH 4 - PROGRESS: at 8.30% examples, 851891 words/s, 

2019-07-14 13:24:11,985 : INFO : worker thread finished; awaiting finish of 6 more threads
2019-07-14 13:24:11,996 : INFO : worker thread finished; awaiting finish of 5 more threads
2019-07-14 13:24:11,998 : INFO : worker thread finished; awaiting finish of 4 more threads
2019-07-14 13:24:11,999 : INFO : worker thread finished; awaiting finish of 3 more threads
2019-07-14 13:24:12,000 : INFO : worker thread finished; awaiting finish of 2 more threads
2019-07-14 13:24:12,009 : INFO : worker thread finished; awaiting finish of 1 more threads
2019-07-14 13:24:12,011 : INFO : worker thread finished; awaiting finish of 0 more threads
2019-07-14 13:24:12,011 : INFO : EPOCH - 5 : training on 41519358 raw words (30350535 effective words) took 27.7s, 1096714 effective words/s
2019-07-14 13:24:12,012 : INFO : training on a 207596790 raw words (151746740 effective words) took 144.2s, 1052544 effective words/s
2019-07-14 13:24:12,012 : WARNING : Effective 'alpha' higher than previous training cycl

2019-07-14 13:25:06,770 : INFO : EPOCH 2 - PROGRESS: at 89.08% examples, 1123378 words/s, in_qsize 19, out_qsize 0
2019-07-14 13:25:07,772 : INFO : EPOCH 2 - PROGRESS: at 92.93% examples, 1123106 words/s, in_qsize 18, out_qsize 1
2019-07-14 13:25:08,781 : INFO : EPOCH 2 - PROGRESS: at 96.71% examples, 1121986 words/s, in_qsize 18, out_qsize 1
2019-07-14 13:25:09,511 : INFO : worker thread finished; awaiting finish of 9 more threads
2019-07-14 13:25:09,517 : INFO : worker thread finished; awaiting finish of 8 more threads
2019-07-14 13:25:09,526 : INFO : worker thread finished; awaiting finish of 7 more threads
2019-07-14 13:25:09,527 : INFO : worker thread finished; awaiting finish of 6 more threads
2019-07-14 13:25:09,528 : INFO : worker thread finished; awaiting finish of 5 more threads
2019-07-14 13:25:09,529 : INFO : worker thread finished; awaiting finish of 4 more threads
2019-07-14 13:25:09,531 : INFO : worker thread finished; awaiting finish of 3 more threads
2019-07-14 13:25:0

2019-07-14 13:26:02,630 : INFO : EPOCH 4 - PROGRESS: at 90.50% examples, 1013014 words/s, in_qsize 19, out_qsize 0
2019-07-14 13:26:03,647 : INFO : EPOCH 4 - PROGRESS: at 94.71% examples, 1020239 words/s, in_qsize 19, out_qsize 0
2019-07-14 13:26:04,651 : INFO : EPOCH 4 - PROGRESS: at 98.94% examples, 1026851 words/s, in_qsize 18, out_qsize 1
2019-07-14 13:26:04,842 : INFO : worker thread finished; awaiting finish of 9 more threads
2019-07-14 13:26:04,856 : INFO : worker thread finished; awaiting finish of 8 more threads
2019-07-14 13:26:04,858 : INFO : worker thread finished; awaiting finish of 7 more threads
2019-07-14 13:26:04,861 : INFO : worker thread finished; awaiting finish of 6 more threads
2019-07-14 13:26:04,862 : INFO : worker thread finished; awaiting finish of 5 more threads
2019-07-14 13:26:04,870 : INFO : worker thread finished; awaiting finish of 4 more threads
2019-07-14 13:26:04,880 : INFO : worker thread finished; awaiting finish of 3 more threads
2019-07-14 13:26:0

2019-07-14 13:26:55,185 : INFO : worker thread finished; awaiting finish of 7 more threads
2019-07-14 13:26:55,190 : INFO : worker thread finished; awaiting finish of 6 more threads
2019-07-14 13:26:55,192 : INFO : worker thread finished; awaiting finish of 5 more threads
2019-07-14 13:26:55,193 : INFO : worker thread finished; awaiting finish of 4 more threads
2019-07-14 13:26:55,204 : INFO : worker thread finished; awaiting finish of 3 more threads
2019-07-14 13:26:55,209 : INFO : worker thread finished; awaiting finish of 2 more threads
2019-07-14 13:26:55,215 : INFO : worker thread finished; awaiting finish of 1 more threads
2019-07-14 13:26:55,220 : INFO : worker thread finished; awaiting finish of 0 more threads
2019-07-14 13:26:55,221 : INFO : EPOCH - 6 : training on 41519358 raw words (30350522 effective words) took 25.3s, 1201083 effective words/s
2019-07-14 13:26:56,231 : INFO : EPOCH 7 - PROGRESS: at 3.53% examples, 1092052 words/s, in_qsize 19, out_qsize 0
2019-07-14 13:26:

2019-07-14 13:27:45,777 : INFO : worker thread finished; awaiting finish of 3 more threads
2019-07-14 13:27:45,778 : INFO : worker thread finished; awaiting finish of 2 more threads
2019-07-14 13:27:45,780 : INFO : worker thread finished; awaiting finish of 1 more threads
2019-07-14 13:27:45,782 : INFO : worker thread finished; awaiting finish of 0 more threads
2019-07-14 13:27:45,783 : INFO : EPOCH - 8 : training on 41519358 raw words (30348177 effective words) took 25.3s, 1199775 effective words/s
2019-07-14 13:27:46,813 : INFO : EPOCH 9 - PROGRESS: at 3.86% examples, 1169595 words/s, in_qsize 19, out_qsize 0
2019-07-14 13:27:47,825 : INFO : EPOCH 9 - PROGRESS: at 7.79% examples, 1185905 words/s, in_qsize 17, out_qsize 2
2019-07-14 13:27:48,830 : INFO : EPOCH 9 - PROGRESS: at 11.28% examples, 1200116 words/s, in_qsize 18, out_qsize 1
2019-07-14 13:27:49,831 : INFO : EPOCH 9 - PROGRESS: at 14.67% examples, 1200180 words/s, in_qsize 17, out_qsize 2
2019-07-14 13:27:50,832 : INFO : EPOC

2019-07-14 13:28:36,044 : INFO : training on a 415193580 raw words (303484384 effective words) took 264.0s, 1149428 effective words/s


(303484384, 415193580)

## Now, let's look at some output 
This first example shows a simple case of looking up words similar to the word `dirty`. All we need to do here is to call the `most_similar` function and provide the word `dirty` as the positive example. This returns the top 10 similar words. 

In [16]:

w1 = "dirty"
model.wv.most_similar (positive=w1)


2019-07-14 13:28:36,059 : INFO : precomputing L2-norms of word weight vectors


[('filthy', 0.8629568815231323),
 ('stained', 0.7823047637939453),
 ('grubby', 0.7788145542144775),
 ('dusty', 0.7734365463256836),
 ('smelly', 0.7699148654937744),
 ('unclean', 0.7689845561981201),
 ('dingy', 0.7349340319633484),
 ('mouldy', 0.7272517681121826),
 ('soiled', 0.7271710634231567),
 ('gross', 0.719116747379303)]

That looks pretty good, right? Let's look at a few more. Let's look at similarity for `polite`, `france` and `shocked`. 

In [17]:
# look up top 6 words similar to 'polite'
w1 = ["polite"]
model.wv.most_similar (positive=w1,topn=6)


[('courteous', 0.9275883436203003),
 ('friendly', 0.8336566686630249),
 ('cordial', 0.8042970895767212),
 ('professional', 0.788394570350647),
 ('attentive', 0.7790228724479675),
 ('curteous', 0.7520910501480103)]

In [18]:
# look up top 6 words similar to 'france'
w1 = ["france"]
model.wv.most_similar (positive=w1,topn=6)


[('canada', 0.6419494152069092),
 ('germany', 0.6231887340545654),
 ('spain', 0.6080336570739746),
 ('england', 0.6063601970672607),
 ('gaulle', 0.5965712666511536),
 ('angeles', 0.59413743019104)]

In [19]:
# look up top 6 words similar to 'shocked'
w1 = ["shocked"]
model.wv.most_similar (positive=w1,topn=6)


[('amazed', 0.7959094643592834),
 ('horrified', 0.7951109409332275),
 ('stunned', 0.778906524181366),
 ('appalled', 0.7667439579963684),
 ('dismayed', 0.7525988817214966),
 ('astonished', 0.7487436532974243)]

That's, nice. You can even specify several positive examples to get things that are related in the provided context and provide negative examples to say what should not be considered as related. In the example below we are asking for all items that *relate to bed* only:

In [20]:
# get everything related to stuff on the bed
w1 = ["bed",'sheet','pillow']
w2 = ['couch']
model.wv.most_similar (positive=w1,negative=w2,topn=10)


[('duvet', 0.7161251306533813),
 ('blanket', 0.709022045135498),
 ('mattress', 0.7010205984115601),
 ('quilt', 0.6725625991821289),
 ('pillowcase', 0.6602649092674255),
 ('matress', 0.6564290523529053),
 ('pillows', 0.6434866189956665),
 ('sheets', 0.637667179107666),
 ('foam', 0.6116433143615723),
 ('satin', 0.6027736663818359)]

### Similarity between two words in the vocabulary

You can even use the Word2Vec model to return the similarity between two words that are present in the vocabulary. 

In [21]:
# similarity between two different words
model.wv.similarity(w1="dirty",w2="smelly")

0.7699148

In [22]:
# similarity between two identical words
model.wv.similarity(w1="dirty",w2="dirty")

1.0

In [23]:
# similarity between two unrelated words
model.wv.similarity(w1="dirty",w2="clean")

0.2670418

Under the hood, the above three snippets computes the cosine similarity between the two specified words using word vectors of each. From the scores, it makes sense that `dirty` is highly similar to `smelly` but `dirty` is dissimilar to `clean`. If you do a similarity between two identical words, the score will be 1.0 as the range of the cosine similarity score will always be between [0.0-1.0]. You can read more about cosine similarity scoring [here](https://en.wikipedia.org/wiki/Cosine_similarity).

### Find the odd one out
You can even use Word2Vec to find odd items given a list of items.

In [24]:
# Which one is the odd one out in this list?
model.wv.doesnt_match(["cat","dog","france"])

/home/geanderson/anaconda3/lib/python3.7/site-packages/gensim/models/keyedvectors.py:876: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


'france'

In [25]:
# Which one is the odd one out in this list?
model.wv.doesnt_match(["bed","pillow","duvet","shower"])


'shower'

## Understanding some of the parameters
To train the model earlier, we had to set some parameters. Now, let's try to understand what some of them mean. For reference, this is the command that we used to train the model.

```
model = gensim.models.Word2Vec (documents, size=150, window=10, min_count=2, workers=10)
```

### `size`
The size of the dense vector to represent each token or word. If you have very limited data, then size should be a much smaller value. If you have lots of data, its good to experiment with various sizes. A value of 100-150 has worked well for me. 

### `window`
The maximum distance between the target word and its neighboring word. If your neighbor's position is greater than the maximum window width to the left and the right, then, some neighbors are not considered as being related to the target word. In theory, a smaller window should give you terms that are more related. If you have lots of data, then the window size should not matter too much, as long as its a decent sized window. 

### `min_count`
Minimium frequency count of words. The model would ignore words that do not statisfy the `min_count`. Extremely infrequent words are usually unimportant, so its best to get rid of those. Unless your dataset is really tiny, this does not really affect the model.

### `workers`
How many threads to use behind the scenes?


## When should you use Word2Vec?

There are many application scenarios for Word2Vec. Imagine if you need to build a sentiment lexicon. Training a Word2Vec model on large amounts of user reviews helps you achieve that. You have a lexicon for not just sentiment, but for most words in the vocabulary. 

Beyond, raw unstructured text data, you could also use Word2Vec for more structured data. For example, if you had tags for a million stackoverflow questions and answers, you could find tags that are related to a given tag and recommend the related ones for exploration. You can do this by treating each set of co-occuring tags as a "sentence" and train a Word2Vec model on this data. Granted, you still need a large number of examples to make it work. 
